In [1]:
import torch
import torch.nn as nn
import torch.optim as optim
import numpy as np

In [40]:
chars = "ABCDEFGHIJKLMNOPQRSTUVWXYZ ?.',[]" #[ : Start, ] : End
toRemember = "I'M RNN, NICE TO MEET YOU!"

def to_one_hot(word):
    start_token = [0] * len(chars)
    start_token[-2] = 1
    start_token = np.array(start_token)
    end_token = [0] * len(chars)
    end_token[-1] = 1
    end_token = np.array(end_token)
    
    out = start_token
    
    for ch in word:
        tmp = [0] * len(chars)
        tmp[chars.index(ch)] = 1
        out = np.vstack([out, tmp])
    out = np.vstack([out, end_token])
    return out
    
def to_letter(one_hot):
    string = ''
    for i in range(len(one_hot)):
        if i != 0 and i != len(one_hot) - 1:
            index = one_hot[i].argmax()
            string += chars[index]
    return string

In [41]:
one_hot = to_one_hot('HELLO')
to_letter(one_hot)

'HELLO'

In [51]:
class RNN(nn.Module):
    def __init__(self, in_size, hid_size, out_size):
        super(RNN, self).__init__()
        self.in_size = in_size
        self.hid_size = hid_size
        self.out_size = out_size
        
        self.i2h = nn.Linear(in_size, hid_size)
        self.h2h = nn.Linear(hid_size, hid_size)
        self.h2o = nn.Linear(hid_size, out_size)
        self.activation = nn.ELU()
        
    def forward(self, x, prev): #?? Don't get it a little
        prev = self.activation(self.i2h(x) + self.h2h(prev))
        out = self.i2o(prev)
        return out, prev
    def first_prev(self):
        return torch.zeros(1, self.hid_size)

In [45]:
net = RNN(len(chars), 100, len(chars))

In [46]:
net

RNN(
  (i2h): Linear(in_features=33, out_features=100, bias=True)
  (h2h): Linear(in_features=100, out_features=100, bias=True)
  (h2o): Linear(in_features=100, out_features=33, bias=True)
  (activation): ELU(alpha=1.0)
)

In [49]:
get_loss = nn.MSELoss()
optimizer = optim.Adam(net.parameters(), lr = 0.01)

In [50]:
n_epoch = 100
for epoch in range(n_epoch):
    optimizer.zero_grad()
    hidden = net.first_prev()
    
    